In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14840, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 14840 (delta 127), reused 154 (delta 110), pack-reused 14655
Receiving objects: 100% (14840/14840), 13.88 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (10210/10210), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 14.4 MB/s 
Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="fXNvqJubpdjp1mf8IS6s")
project = rf.workspace("detection-axsgy").project("uav-ce0zg")
dataset = project.version(7).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 145 kB 22.2 MB/s 
     |████████████████████████████████| 178 kB 63.2 MB/s 
     |████████████████████████████████| 138 kB 70.6 MB/s 
     |████████████████████████████████| 54 kB 4.1 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
     |████████████████████████████████| 62 kB 2.1 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=521a3fb0301e906310fedcd271b36dc67fabdb3a771fa2334c3a5eada72307b6
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certi

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Uav-7 in yolov5pytorch:: 100%|██████████| 10042/10042 [00:02<00:00, 3943.16it/s]


In [ ]:
%cd /content/yolov5
#after following the link above, recieve python code with these fields filled in
#from roboflow import Roboflow
#rf = Roboflow(api_key="YOUR API KEY HERE")
#project = rf.workspace().project("YOUR PROJECT")
#dataset = project.version("YOUR VERSION").download("yolov5")
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat //content/yolov5/models/yolov5s.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
#Anchor boxes are a set of predefined bounding boxes of a certain height and width. These boxes are defined to capture the scale and 
#aspect ratio of specific object classes you want to detect and are typically chosen based on object sizes in your training datasets
# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32


# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  #The focus layer was created to reduce layers, parameters, FLOPS, and CUDA memory and improve forward and backward speed while minimizing the impact of mAP
  ## ch_in, ch_out, kernel, stride, padding, groups
  [[-1, 1, Focus, [256, 3]],  # 0-P1/2
   # Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)
   [-1, 1, Conv, [256, 3, 2]],  # 1-P2/4
   #It employs a CSPNet strategy to partition the feature map of the base layer into two parts 
   #and then merges them through a cross-stage hierarchy. The use of a split and merge strategy allows for more gradient flow through the network.
   # ch_in, ch_out, number, shortcut, groups, expansion
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [256, 3, 2]],  # 7-P5/32
   #Spatial Pyramid Pooling (SPP) is a pooling layer that removes the fixed-size constraint of the network, i.e. a CNN does not require a fixed-size input image
   [-1, 1, SPP, [256, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [128, False]],  # 13
 #nn.Upsample(scale_factor=2, mode='nearest')
   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [128, False]],  # 17 (P3/medium)

   [-1, 1, Conv, [128, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [128, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [128, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [128, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 32 --epochs 200 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name FeatureNetsmall_results  --cache

# Evaluate Custom YOLOv5 Detector Performance

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/FeatureNetsmall_results/results.png', width=1000)  # view results.png

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/
%ls runs/train/FeatureNetsmall_results/weights

In [ ]:

%cd /content/yolov5/
!python detect.py --weights runs/train/FeatureNetsmall_results/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize

/content/yolov5
detect: weights=['/content/yolov5/mfnets.pt'], source=/content/yolov5/Uav-7/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=True, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 2789023 parameters, 0 gradients, 18.9 GFLOPs
Saving runs/detect/exp/0046_jpg.rf.2dd8c3d58548003a34f42844ecd0feb6/stage0_Focus_features.png... (32/128)
Saving runs/detect/exp/0046_jpg.rf.2dd8c3d58548003a34f42844ecd0feb6/stage1_Conv_features.png... (32/128)
Saving runs/detect/exp/0046_jpg.rf.2dd8c3d58548003a34f42844ecd0feb6/stage2_BottleneckCSP_features.png... (32/64)
Savi

In [ ]:
#### Visulize each layers feature map
#######Run this cell to know the whats inside every lare feature
#### stop it ur self after one image is done
## Print whole folder feature maps and save them
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize
####### Only single image
!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77.jpg --visualize
                                               

In [ ]:
%cd /content/yolov5/
!python val.py --weights /content/yolov5/mfnets.pt --data {dataset.location}/data.yaml --iou 0.3 --img 416 --verbose --save-txt

/content/yolov5
val: data=/content/yolov5/Uav-7/data.yaml, weights=['/content/yolov5/mfnets.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.3, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 2789023 parameters, 0 gradients, 18.9 GFLOPs
val: Scanning '/content/yolov5/Uav-7/valid/labels.cache' images and labels... 415 found, 0 missing, 0 empty, 0 corrupt: 100% 415/415 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 13/13 [00:06<00:00,  2.01it/s]
                 all        415        876        378       44.5      0.909      0.877      0.893      0.899      0.488
                b

In [ ]:
!python val.py --task study --weights /content/yolov5/mfnets.pt --data {dataset.location}/data.yaml --iou 0.3

val: data=/content/yolov5/Uav-7/data.yaml, weights=['/content/yolov5/mfnets.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.3, task=study, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False

Running study_data_mfnets.txt --imgsz 256...
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 2789023 parameters, 0 gradients, 18.9 GFLOPs
val: Scanning '/content/yolov5/Uav-7/valid/labels.cache' images and labels... 415 found, 0 missing, 0 empty, 0 corrupt: 100% 415/415 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 13/13 [00:03<00:00,  3.82it/s]
                 all        415        876        346         50      0.892      0.826      0.857      

In [ ]:
!python val.py --task speed --weights /content/yolov5/mfnets.pt --data {dataset.location}/data.yaml --iou 0.3

val: data=/content/yolov5/Uav-7/data.yaml, weights=['/content/yolov5/mfnets.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.3, task=speed, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 2789023 parameters, 0 gradients, 18.9 GFLOPs
val: Scanning '/content/yolov5/Uav-7/valid/labels.cache' images and labels... 415 found, 0 missing, 0 empty, 0 corrupt: 100% 415/415 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 13/13 [00:04<00:00,  2.90it/s]
                 all        415        876      0.774       0.78      0.813      0.388
                bird        415        587      0.805      0.872      0.886      0.407
        

In [ ]:
!python val.py --weights /content/yolov5/mfnets.pt --data {dataset.location}/data.yaml --iou 0.3 --img 416 --verbose --save-txt

val: data=/content/yolov5/Uav-7/data.yaml, weights=['/content/yolov5/mfnets.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.3, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 2789023 parameters, 0 gradients, 18.9 GFLOPs
val: Scanning '/content/yolov5/Uav-7/valid/labels.cache' images and labels... 415 found, 0 missing, 0 empty, 0 corrupt: 100% 415/415 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 13/13 [00:06<00:00,  2.03it/s]
                 all        415        876        378       44.5      0.909      0.877      0.893      0.899      0.488
                bird        415  

In [ ]:
##I have created a zip file:
!zip -r /content/FeatureNet-smal.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/FeatureNet-smal.zip")

In [ ]:
##I have created a zip file:
!zip -r /content/detectfeatures.zip /content/yolov5/runs/detect/exp/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/detectfeatures.zip")

  adding: content/yolov5/runs/detect/exp/ (stored 0%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77/ (stored 0%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77/stage2_BottleneckCSP_features.png (deflated 0%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77/stage7_Conv_features.png (deflated 33%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77/stage8_SPP_features.npy (deflated 8%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77/stage0_Focus_features.png (deflated 1%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77/stage13_BottleneckCSP_features.png (deflated 15%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77/stage3_Conv_features.npy (deflated 8%)
  adding: content/yolov5/runs/detect/exp/0094_jpg.rf.09856e96b7f9f77c2bc5024

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>